In [91]:
import pandas as pd
import json
import re
from nltk.corpus import wordnet as wn


In [92]:
def json_file_to_df(filename):
    # Leggi il file JSON
    with open(filename, 'r') as file:
        data = file.read()
    # Parsifica il JSON in un DataFrame
    data = json.loads(data)
    # Converte gli oggetti non di tipo Series in Series
    for key, value in data.items():
        if not isinstance(value, pd.Series):
            data[key] = pd.Series(value)

    # Crea il DataFrame
    df = pd.DataFrame(data)
    return df

def elaborate_dataset(dataset):
    result = []
    for synset in dataset[:-2]:
        result.append([synset.split(":")[0].replace("Synset('","").replace("')",""),0])
    return result

def calc_basicness_score(dataset,df):
    for index, tag in enumerate(df[:-2]):
        if tag == "basic":
            dataset[index][1] += 1
    return dataset

def calc_avg(dataset, test_number):
     for index, score in enumerate(dataset):
         dataset[index][1] = dataset[index][1]/test_number
     return dataset

In [93]:
df = json_file_to_df("dataset_basic_advanced_TLN2023/1.json")
dataset = elaborate_dataset(df["dataset"])
dataset = calc_basicness_score(dataset,df["answers"])
dataset = calc_basicness_score(dataset,json_file_to_df("dataset_basic_advanced_TLN2023/2.json")["answers"])
dataset = calc_basicness_score(dataset,json_file_to_df("dataset_basic_advanced_TLN2023/3.json")["answers"])
dataset = calc_basicness_score(dataset,json_file_to_df("dataset_basic_advanced_TLN2023/4.json")["answers"])
dataset = calc_basicness_score(dataset,json_file_to_df("dataset_basic_advanced_TLN2023/5.json")["answers"])
dataset = calc_basicness_score(dataset,json_file_to_df("dataset_basic_advanced_TLN2023/6.json")["answers"])
dataset = calc_basicness_score(dataset,json_file_to_df("dataset_basic_advanced_TLN2023/7.json")["answers"])
dataset = calc_basicness_score(dataset,json_file_to_df("dataset_basic_advanced_TLN2023/8.json")["answers"])
dataset = calc_basicness_score(dataset,json_file_to_df("dataset_basic_advanced_TLN2023/9.json")["answers"])
dataset = calc_basicness_score(dataset,json_file_to_df("dataset_basic_advanced_TLN2023/10.json")["answers"])
dataset = calc_avg(dataset,10)

print(dataset)


[['war.n.01', 1.0], ['fiefdom.n.01', 0.0], ['bed.n.03', 1.0], ['return_on_invested_capital.n.01', 0.0], ['texture.n.02', 0.8], ['news.n.01', 1.0], ['look.n.02', 1.0], ['caddy.n.01', 0.2], ['weeder.n.01', 0.0], ['avenue.n.02', 0.4], ['adar.n.01', 0.0], ['bedtime.n.01', 0.8], ['inversion.n.08', 0.7], ['yak.n.01', 0.1], ['breath.n.05', 0.9], ['executive_clemency.n.01', 0.0], ['muse.n.02', 0.3], ['effect.n.06', 1.0], ['quickening.n.02', 0.1], ['sleeper.n.09', 0.5], ['caravanning.n.01', 0.0], ['jotter.n.01', 0.0], ['armageddon.n.02', 0.2], ['compass_point.n.01', 0.1], ['blackwater_fever.n.01', 0.0], ['respect.n.03', 1.0], ['position.n.06', 1.0], ['message.n.02', 1.0], ['arrest.n.02', 0.8], ['motivation.n.01', 0.9], ['day.n.04', 1.0], ['nose_cone.n.01', 0.0], ['discussion.n.02', 0.9], ['glow.n.05', 0.5], ['alcalde.n.01', 0.0], ['draft_board.n.01', 0.2], ['multitude.n.03', 0.4], ['hour.n.02', 1.0], ['book.n.02', 1.0], ['degree.n.02', 1.0], ['show-stopper.n.01', 0.0], ['military_position.n.01'

In [94]:
def normalize(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

def extract_synset_features(synset):
    # Lunghezza della descrizione
    desc_length = normalize(len(synset.definition()),0,300)

    # Numero di esempi
    num_examples = normalize(len(synset.examples()),0,15)

    # Altezza dell'albero WordNet
    wd_height = normalize(synset.max_depth(),0,20)

    # Lunghezza della parola
    word_length = normalize(len(synset.lemmas()[0].name()),1,30)

    # Lunghezza media degli esempi
    examples_avg_length = normalize((sum(len(example) for example in synset.examples()) / num_examples if num_examples > 0 else 0),0,300)

    # Numero di padri
    num_parents = normalize(len(synset.hypernym_paths()[0]) - 1,0,10)

    # Numero di figli
    num_children = normalize(len(synset.hyponyms()),0,30)

    return [desc_length, num_examples, wd_height, word_length, examples_avg_length, num_parents, num_children]

def extract_fetures_from_dataset(dataset):
    for index, score in enumerate(dataset):
        synset = wn.synset(dataset[index][0])
        dataset[index].append(extract_synset_features(synset))

    return dataset


In [95]:
dataset = extract_fetures_from_dataset(dataset)
print(dataset)

[['war.n.01', 1.0, [0.15, 0.06666666666666667, 0.35, 0.06896551724137931, 2.1, 0.7, 0.3]], ['fiefdom.n.01', 0.0, [0.12666666666666668, 0.0, 0.3, 0.20689655172413793, 0.0, 0.6, 0.0]], ['bed.n.03', 1.0, [0.17666666666666667, 0.06666666666666667, 0.25, 0.06896551724137931, 2.05, 0.5, 0.13333333333333333]], ['return_on_invested_capital.n.01', 0.0, [0.6766666666666666, 0.0, 0.3, 0.8620689655172413, 0.0, 0.6, 0.0]], ['texture.n.02', 0.8, [0.11333333333333333, 0.06666666666666667, 0.45, 0.20689655172413793, 1.5, 0.9, 0.0]], ['news.n.01', 1.0, [0.15, 0.06666666666666667, 0.25, 0.10344827586206896, 1.6, 0.5, 0.1]], ['look.n.02', 1.0, [0.24333333333333335, 0.26666666666666666, 0.4, 0.10344827586206896, 1.4875, 0.8, 0.36666666666666664]], ['caddy.n.01', 0.2, [0.07, 0.0, 0.4, 0.13793103448275862, 0.0, 0.8, 0.0]], ['weeder.n.01', 0.0, [0.10666666666666667, 0.0, 0.65, 0.1724137931034483, 0.0, 1.0, 0.0]], ['avenue.n.02', 0.4, [0.09666666666666666, 0.0, 0.45, 0.1724137931034483, 0.0, 0.9, 0.0]], ['ada